In [2]:
# 셀레니움 모듈 설치
# 별도 터미널에서 진행
# https://selenium-python.readthedocs.io/
# !conda install selenium -y

^C


# 웹 드라이버 가동및 지점 정보 접근 루트 확보

In [5]:
import selenium

# 라이브러리 버전 체크 변수
selenium.__version__

'3.141.0'

In [6]:
# 셀레니움 모듈 가져오기(웹드라이버만)
from selenium import webdriver as wd
# 시간 
import time

In [7]:
# 웹드라이버 프로그램 준비
# 통상 크롬을 사용한다면, 개발 PC에 설치된 크롬 버전과 동일한 웹드라이버 획득
# 현재 108.0.5359.125(공식 빌드) (64비트)
# https://chromedriver.chromium.org/downloads 접속 
# 웹드라이버 가동 (상대경로 않되면 절대경로로 변경해서 시도)
# driver = wd.Chrome('./chromedriver.exe')

driver = wd.Chrome('C://Users//USER//Desktop//Py_Projects//selenium_level4//chromedriver.exe')

In [8]:
# 접속
# 사이트가 로딩되는 속도가 천차 만별이다
# 로딩이 완료되었는지 검사하는 루틴이 필요 -> 암묵적 대기
# 귀찮으면 명시적으로 특정 시간을 대기하면 된다
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

# 강제로 10초 대기(명시적 대기)
time.sleep(1*5)

In [9]:
# 시도를 특정한다
# 주관적으로 수행했는데 반복성이 않보여서, 기계적으로 생성
# 세레니움 css selector가 간혹 잘 않될때도 있어서 잘 되는 표현으로 사용
'''
div.loca_step1_cont > ul > li:nth-child(1) > a
'''
# 서울 클릭
css_sel = 'div.loca_step1_cont > ul > li:nth-child(1) > a'
si_do_a_tag = driver.find_element_by_css_selector( css_sel )
si_do_a_tag.click()

time.sleep(1*3)

In [10]:
# 시군구 클릭
# 서울 > 중구 클릭 실행 => 전체가 있다 => 전체 클릭으로 변경
# #mCSB_2_container > ul > li:nth-child(1) > a
css_sel = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector( css_sel ).click()

time.sleep(1*3)

- 최족 목적 페이지까지 로드되면, 셀레이눔으로 처리할지, BS4로 처리할지 결정 필요

# 셀레니움으로 지점 정보 추출

In [11]:
# 셀레니움
len( driver.find_elements_by_css_selector('#mCSB_3_container > ul > li') )

598

# BeautifulSoup로 지점 정보 추출

In [12]:
# BS4
from bs4 import BeautifulSoup
src  = driver.page_source # 현재 페이지의 HTML 소스
soup = BeautifulSoup( src, 'html5lib' )

In [14]:
len( soup.select('.quickSearchResultBoxSidoGugun > li') )

598

In [30]:
# 좌우 공백은 내부적으로 자동처리해줌 => strip()은 미사용해도됨
float( '37.501087   '  ), #float( None  )

(37.501087,)

In [32]:
# 추출 정보 : 지점명, 위도, 경도, 주소
starbucks_store_infos = list()
for li in soup.select('.quickSearchResultBoxSidoGugun > li'):
    starbucks_store_infos.append({
        'name': li.get('data-name') , # 지점명, 속성값 추출 => 요소.get('속성이름')
        'lat' : float( li.get('data-lat') ) ,   # 위도 -> float()
        'long': float( li.get('data-long')) ,   # 경도 -> float() 
        'code': li.get('data-code') , # 지점코드
        'storecd': li.get('data-storecd') ,                 # 관리코드(?)
        'addr': li.p.text.strip()[:len('1522-3232')*-1] ,   # 전화번호가 모두 동일하다
        'spec': li.i.get('class')[0][len('pin_'):]  # class는 속성값으로 뽑으면 리스트로 나온다(특징), pin_ 제거    
    })
    #break
len( starbucks_store_infos ),starbucks_store_infos[0]

(598,
 {'name': '역삼아레나빌딩',
  'lat': 37.501087,
  'long': 127.043069,
  'code': '3762',
  'storecd': '1509',
  'addr': '서울특별시 강남구 언주로 425 (역삼동)',
  'spec': 'general'})

In [33]:
starbucks_store_infos = [ {
        'name': li.get('data-name') , # 지점명, 속성값 추출 => 요소.get('속성이름')
        'lat' : float( li.get('data-lat') ) ,   # 위도 -> float()
        'long': float( li.get('data-long')) ,   # 경도 -> float() 
        'code': li.get('data-code') , # 지점코드
        'storecd': li.get('data-storecd') ,                 # 관리코드(?)
        'addr': li.p.text.strip()[:len('1522-3232')*-1] ,   # 전화번호가 모두 동일하다
        'spec': li.i.get('class')[0][len('pin_'):]  # class는 속성값으로 뽑으면 리스트로 나온다(특징), pin_ 제거    
} for li in soup.select('.quickSearchResultBoxSidoGugun > li') ]

len(starbucks_store_infos), starbucks_store_infos[-1]

(598,
 {'name': '중화역',
  'lat': 37.60170912407773,
  'long': 127.07841136432036,
  'code': '3801',
  'storecd': '1749',
  'addr': '서울특별시 중랑구 봉화산로 35 ',
  'spec': 'general'})

# 전국 지점 데이터 획득

- 전개 순서
    - 시도 개수만큼 반복(17) => 사전 조사를 통해서 반복횟수를 결정했다
        - https://www.starbucks.co.kr/store/store_map.do?disp=locale 접속
            - 난수*5초대기
        - 시도 선택
            - 난수*5초대기
        - 전체 선택
            - 난수*5초대기
        - 추출
            - 난수*1초대기
    - 난수를 사용해서 대기시간을 미세하게 조정해주면, 반복적으로 데이터 수집해 상대방 사이트에서 패턴이 잡히지 않는다

In [41]:
import random

term_make = lambda x, y:random.randint(x,y)*0.1 # 0.1 ~ 0.9
1*5 + term_make(1,5)

5.1

In [47]:
from bs4 import BeautifulSoup

# 모든 지점의 정보를 담는 그릇
starbucks_store_infos = list()

for sido_idx in range(1, 17+1):
    # 초기 진입 사이트 접속
    driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')
    time.sleep(1*5 + term_make(1,5))

    # 시도 선택
    css_sel     = f'div.loca_step1_cont > ul > li:nth-child({sido_idx}) > a'
    si_do_a_tag = driver.find_element_by_css_selector( css_sel )
    si_do_a_tag.click()
    time.sleep(1*5 + term_make(1,5))

    # 전체 선택, 1번 맴버가 무조건 전체이다 -> 고정
    css_sel     = '#mCSB_2_container > ul > li:nth-child(1) > a'
    driver.find_element_by_css_selector( css_sel ).click()
    # 로딩 속도에 따라 반응이 다르게 온다 => 보수적으로 10초 결정
    time.sleep(1*10 + term_make(1,5))

    # soup(DOM Tree) 생성
    src  = driver.page_source # 현재 페이지의 HTML 소스
    soup = BeautifulSoup( src, 'html5lib' )
    # 정보추출
    starbucks_store_local_infos = [ {
        'name': li.get('data-name') , # 지점명, 속성값 추출 => 요소.get('속성이름')
        'lat' : float( li.get('data-lat') ) ,   # 위도 -> float()
        'long': float( li.get('data-long')) ,   # 경도 -> float() 
        'code': li.get('data-code') , # 지점코드
        'storecd': li.get('data-storecd') ,                 # 관리코드(?)
        'addr': li.p.text.strip()[:len('1522-3232')*-1] ,   # 전화번호가 모두 동일하다
        'spec': li.i.get('class')[0][len('pin_'):]  # class는 속성값으로 뽑으면 리스트로 나온다(특징), pin_ 제거    
    } for li in soup.select('.quickSearchResultBoxSidoGugun > li') ]

    # starbucks_store_infos에 starbucks_store_local_infos값 하나하나를 맴버로 추가한다
    starbucks_store_infos.extend( starbucks_store_local_infos )

    # 샘플 1개만 작동잘되는지 체크
    if sido_idx==2:
        break

len(starbucks_store_infos) # 598 + 418

1016

In [46]:
a = []
b = [1,2]
c = [3,4]
#a.append(b) # [[1, 2]]
a.extend(b)  # [1, 2]
a.extend(c)  # [1, 2, 3, 4]
a


[1, 2, 3, 4]

In [51]:
# df 변환
import pandas as pd

df = pd.DataFrame.from_dict( starbucks_store_infos )
df.head(2)

,name,lat,long,code,storecd,addr,spec
0,역삼아레나빌딩,37.501087,127.043069,3762,1509,서울특별시 강남구 언주로 425 (역삼동),general
1,논현역사거리,37.510178,127.022223,3672,1434,서울특별시 강남구 강남대로 538 (논현동),general


In [52]:
# csv 저장 : dataFrame -> csv 저장
df.to_csv('starbucks_store.csv', index=False)

In [53]:
df.to_excel('starbucks_store.xlsx', index=False)

In [48]:
# 브라우저 닫기
driver.close()
driver.quit()

# 파이썬 파일에서 사용
# import sys
# sys.exit(0)